
# 🌼 RAG기법의 이해와 적용 - 3차시(24.12.02)

---

In [2]:
from langchain.memory import ConversationBufferMemory
# 메모리에 input으로 우리의 응답, output으로 AI의 응답을 담을 수 있다
# FIFO

memory = ConversationBufferMemory()

C:\Users\RMARKET\AppData\Local\Temp\ipykernel_2404\3559429672.py:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


In [3]:
memory.save_context(
    inputs = {
        'human': '안녕하세요, 휴대폰을 구매하러 왔습니다.'
    },
    outputs = {
        'ai': '안녕하세요, 새 휴대폰을 구매하신다니 기쁩니다.'
    }
)

In [4]:
memory.load_memory_variables({})

{'history': 'Human: 안녕하세요, 휴대폰을 구매하러 왔습니다.\nAI: 안녕하세요, 새 휴대폰을 구매하신다니 기쁩니다.'}

In [5]:
from dotenv import load_dotenv

load_dotenv()

True

In [6]:
from langchain_teddynote import logging
logging.langsmith("langchain_practice")

LangSmith 추적을 시작합니다.
[프로젝트명]
langchain_practice


In [7]:
from operator import itemgetter
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI

In [39]:
model = ChatOpenAI(model='gpt-4o')

In [40]:
prompt = ChatPromptTemplate.from_messages(
    [
        ('system', '너는 친절한 챗봇이야'),
        MessagesPlaceholder(variable_name='chat_history'),  # AI 응답 자리 만들어주기
        ('human', '{input}')  # 동적으로 처리하기 위한 처리
    ]
)

In [77]:
memory = ConversationBufferMemory(return_messages=True, memory_key='chat_history')

In [78]:
memory.load_memory_variables({})

{'chat_history': []}

In [79]:
runnable = RunnablePassthrough.assign(
    chat_history = RunnableLambda(memory.load_memory_variables) | itemgetter('chat_history')
    # 메모리의 대화 내용을 로드하고, 메모리의 대화 기록인 chat_history(key)만 get
    # 전체 입력 데이터에서 chat_history만 가져오기
)

In [80]:
chain = runnable | prompt | model

In [81]:
response = chain.invoke({'input': '만나서 반갑습니다. 제 이름은 수빈입니다.'})
print(response)

content='안녕하세요, 수빈님! 만나서 반가워요. 어떻게 도와드릴까요?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 36, 'total_tokens': 57, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_831e067d82', 'finish_reason': 'stop', 'logprobs': None} id='run-2b7f0c46-0ded-4c1e-a127-28d4de3e9eb1-0' usage_metadata={'input_tokens': 36, 'output_tokens': 21, 'total_tokens': 57, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [82]:
memory.save_context({'human': '만나서 반갑습니다. 제 이름은 수빈입니다.'}, {'ai': response.content})

In [83]:
response = chain.invoke({'input': '제 이름이 무엇이었는지 기억하시나요?'})

In [84]:
memory.save_context({'human': '제 이름이 무엇이었는지 기억하시나요?'}, {'ai': response.content})

In [85]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='만나서 반갑습니다. 제 이름은 수빈입니다.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='안녕하세요, 수빈님! 만나서 반가워요. 어떻게 도와드릴까요?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='제 이름이 무엇이었는지 기억하시나요?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='네, 수빈님이라고 하셨죠. 다른 질문이나 도움이 필요하시면 언제든지 말씀해 주세요!', additional_kwargs={}, response_metadata={})]}

In [63]:
from operator import itemgetter
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.memory import ConversationBufferWindowMemory

In [71]:
memory = ConversationBufferWindowMemory(k=1, return_messages=True, memory_key='chat_history')
# k: 몇 개의 기록을 기억할 것인지
# return_messages: 메모리에서 반환되는 형식을 개별 대화 형식으로 (False: 텍스트 형식으로 반환)

C:\Users\RMARKET\AppData\Local\Temp\ipykernel_2404\1883192381.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=1, return_messages=True, memory_key='chat_history')


In [ ]:
def conversation_chain(memory, query):
    chat_history = RunnablePassthrough.assign(
        chat_history = RunnableLambda(memory.load_memory_variables) | itemgetter(memory.memory_key)
    )
    llm = ChatOpenAI(model='gpt-4o', temperature=0)
    prompt = ChatPromptTemplate.from_messages(
        [
            ('system', '너는 친절한 AI 비서야'),
            MessagesPlaceholder(variable_name='chat_history'),
            ('human', '{input}')
        ]
    )
    chain = chat_history | prompt | llm | StrOutputParser()
    # StrOutputParser: 출력되는 값을 자동으로 파싱 (불필요한 개행문자, 특수기호 등을 자동으로 처리)

    answer = chain.invoke({'input': query})
    memory.save_context(inputs={'human': query}, outputs= {'ai': answer})
    return answer

In [73]:
response = conversation_chain(
    memory = memory,
    query = '갑자기 날씨가 너무 추워졌어. 이 때 먹을만한 음식을 추천해줘'
)
print('ai : ', response)

ai :  추운 날씨에는 몸을 따뜻하게 해줄 수 있는 음식을 먹는 것이 좋습니다. 다음과 같은 음식을 추천해드릴게요:

1. **김치찌개**: 매콤하고 뜨거운 국물이 몸을 따뜻하게 해줍니다. 돼지고기나 두부를 넣어 더욱 풍성하게 즐길 수 있어요.

2. **된장찌개**: 구수한 맛의 된장찌개는 다양한 채소와 두부를 넣어 영양가도 높고, 속을 든든하게 해줍니다.

3. **삼계탕**: 닭고기와 인삼, 대추 등을 넣어 끓인 삼계탕은 몸을 따뜻하게 하고 기운을 북돋아 줍니다.

4. **칼국수**: 뜨거운 국물에 쫄깃한 면발이 어우러진 칼국수는 추운 날씨에 제격입니다. 해물이나 닭고기를 넣어 다양한 맛을 즐길 수 있어요.

5. **호떡**: 달콤한 시럽이 들어간 호떡은 간식으로 좋고, 따뜻하게 먹으면 기분도 좋아집니다.

6. **붕어빵**: 겨울철 길거리 간식으로 인기 있는 붕어빵은 따뜻하고 달콤한 팥소가 매력적입니다.

이 외에도 따뜻한 차나 코코아를 함께 마시면 더욱 포근한 느낌을 받을 수 있을 거예요.


In [74]:
response = conversation_chain(
    memory = memory,
    query = '그럼 이럴 때 들을만한 노래를 추천해줘'
)
print('ai : ', response)

ai :  추운 날씨에 듣기 좋은 노래는 따뜻한 분위기를 느낄 수 있는 곡들이 좋습니다. 다음과 같은 노래들을 추천해드릴게요:

1. **아이유 - "밤편지"**: 잔잔하고 감성적인 멜로디가 마음을 따뜻하게 해줍니다.

2. **폴킴 - "모든 날, 모든 순간"**: 부드러운 목소리와 감미로운 멜로디가 잘 어우러진 곡입니다.

3. **김광석 - "어느 60대 노부부 이야기"**: 감성적인 가사와 멜로디가 추운 날씨에 잘 어울립니다.

4. **에픽하이 - "춥다 (feat. 이하이)"**: 제목처럼 추운 날씨에 듣기 좋은 곡으로, 감성적인 랩과 보컬이 인상적입니다.

5. **존 레전드 - "All of Me"**: 따뜻한 사랑의 감정을 느낄 수 있는 발라드 곡입니다.

6. **Sarah McLachlan - "Winter Song"**: 겨울의 분위기를 잘 담아낸 곡으로, 차분하게 듣기 좋습니다.

이 노래들이 추운 날씨에 조금이나마 따뜻함을 전해줄 수 있기를 바랍니다. 따뜻한 음료와 함께 감상해보세요!


In [75]:
response = conversation_chain(
    memory = memory,
    query = '아까 이야기했던 음식들 중에서 첫 번째 음식의 레시피를 알려줘'
)
print('ai : ', response)

ai :  죄송하지만 이전에 음식에 대한 이야기를 나눈 적이 없는 것 같습니다. 어떤 음식을 말씀하시는지 알려주시면 그에 대한 레시피를 제공해드리겠습니다. 특정한 요리를 원하신다면 말씀해 주세요!


In [76]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='아까 이야기했던 음식들 중에서 첫 번째 음식의 레시피를 알려줘', additional_kwargs={}, response_metadata={}),
  AIMessage(content='죄송하지만 이전에 음식에 대한 이야기를 나눈 적이 없는 것 같습니다. 어떤 음식을 말씀하시는지 알려주시면 그에 대한 레시피를 제공해드리겠습니다. 특정한 요리를 원하신다면 말씀해 주세요!', additional_kwargs={}, response_metadata={})]}

In [86]:
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory

In [88]:
llm = ChatOpenAI(model='gpt-4o-mini')

In [89]:
memory = ConversationSummaryBufferMemory(
    llm = llm,
    max_token_limit= 200,
    return_messages= True
)

C:\Users\RMARKET\AppData\Local\Temp\ipykernel_2404\418450421.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryBufferMemory(


In [90]:
memory.save_context(
    inputs={"human": "이 식당에서 가장 인기 있는 메뉴는 무엇인가요?"},
    outputs={
        "ai": "이 식당에서 가장 인기 있는 메뉴는 불고기 정식, 해물파전, 비빔밥, 그리고 감자탕입니다. 특히 불고기는 달콤하고 짭짤한 맛으로 외국인 손님들에게도 큰 인기를 끌고 있습니다."
    },
)
memory.save_context(
    inputs={"human": "채식주의자를 위한 메뉴가 제공되나요?"},
    outputs={
        "ai": "네, 채식주의자를 위한 메뉴로 채소 비빔밥, 두부구이, 야채전, 그리고 나물 반찬 세트를 제공합니다. 신선한 제철 채소로 만들어져 건강하고 맛있는 식사를 즐기실 수 있습니다."
    },
)
memory.save_context(
    inputs={"human": "어린이를 위한 메뉴도 있나요?"},
    outputs={
        "ai": "네, 어린이를 위한 메뉴로 미니 김밥, 떡볶이, 그리고 달콤한 간장 치킨을 준비하고 있습니다. 아이들이 좋아할 만한 맛과 건강을 고려한 요리들입니다."
    },
)
memory.save_context(
    inputs={"human": "이 식당은 어떤 분위기를 가지고 있나요?"},
    outputs={
        "ai": "이 식당은 한옥 스타일의 인테리어로 전통적인 한국의 분위기를 느낄 수 있습니다. 따뜻하고 편안한 조명과 현대적인 디자인이 조화를 이루어 가족 단위 손님뿐만 아니라 연인들의 데이트 장소로도 인기가 많습니다."
    },
)

In [93]:
memory.load_memory_variables({})['history']

[SystemMessage(content='The human asks what the most popular menu items are at the restaurant. The AI responds that the most popular items are bulgogi, seafood pancake, bibimbap, and potato soup, noting that bulgogi is particularly favored by foreign guests for its sweet and salty flavor. The human then inquires about vegetarian options, and the AI confirms that they offer vegetable bibimbap, grilled tofu, vegetable pancakes, and a set of seasoned vegetables made with fresh seasonal produce for a healthy and delicious meal.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='어린이를 위한 메뉴도 있나요?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='네, 어린이를 위한 메뉴로 미니 김밥, 떡볶이, 그리고 달콤한 간장 치킨을 준비하고 있습니다. 아이들이 좋아할 만한 맛과 건강을 고려한 요리들입니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='이 식당은 어떤 분위기를 가지고 있나요?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='이 식당은 한옥 스타일의 인테리어로 전통적인 한국의 분위기를 느낄 수 있습니다. 따뜻하고 편안한 조명과 현대적인 디자인이 조화를 

In [94]:
memory.load_memory_variables({})['history'][0].content

'The human asks what the most popular menu items are at the restaurant. The AI responds that the most popular items are bulgogi, seafood pancake, bibimbap, and potato soup, noting that bulgogi is particularly favored by foreign guests for its sweet and salty flavor. The human then inquires about vegetarian options, and the AI confirms that they offer vegetable bibimbap, grilled tofu, vegetable pancakes, and a set of seasoned vegetables made with fresh seasonal produce for a healthy and delicious meal.'